# Inštalácia knižníc

https://tfhub.dev/google/universal-sentence-encoder/2

In [1]:
!pip install -U sentence-transformers
!pip install BERTopic


  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311706 sha256=374021762eafd4f5a88bcc8fe1dd08c3fee9d217884af38becf3c89a27cdae17
  Stored in directory: /root/.cache/pip/wheels/42/63/fb/314ad6c3b270887a3ecb588b8e5aac50b0fad38ff89bb6dff2
Successfully built hdbscan
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=3f8e36b3b8c482b0d9adbc1b1c825e4b998a35f2f2ad46fa6536ebf53f79d7dd
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.2-cp37-none-any.whl size=51351 sha256=a568907075648167fc1a5c14262207e82451f449a15fd2dc9a317d88a79092b2
  Stored in directory: /root/.cache/pip/wheels/ba/52/4e/4c28d04d144a28f89e2575fb63628df6e6d49b56c5ddd0c74e
Successfully built umap-learn pynndescent
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascienc

In [2]:
from sklearn.utils import class_weight
from tqdm import tqdm

from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig


import os
import pandas as pd
import numpy as np

In [3]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 4.9MB/s 


In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pickle

In [5]:
from transformers import AutoModel, BertTokenizerFast


In [6]:
from keras.layers.recurrent import LSTM, RNN
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.layers import Dense, Activation,GRU, SimpleRNN
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LambdaCallback
from tensorflow.keras.layers import Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
from tensorflow.keras.layers import Input,Bidirectional,GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate,LeakyReLU
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences


In [8]:
from bertopic import BERTopic

In [ ]:
!pip install datasets

# Tweet BERT

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:


path_data = '/content/drive/My Drive/'
path = os.path.join(path_data, 'data_preprocessed.csv')
datatobert = pd.read_csv(path,encoding='latin-1')
data = pd.read_csv(path,encoding='latin-1') #,names=['Sentiment', 'News']


# with open("/content/drive/My Drive/Test_Twitter_Sentences", 'rb') as ff:
#   testlistofSentences = pickle.load(ff)
#   ff.close()

path = os.path.join(path_data, 'tweetsdataset.csv')
testdata = pd.read_csv(path)
testdata=testdata.drop(columns=['Unnamed: 0'])

In [11]:
path_data = '/content/drive/My Drive/'
path = os.path.join(path_data, 'BERT_senti_data.csv')
bertdata = pd.read_csv(path,encoding='latin-1')

In [ ]:
datatobert=pd.concat([datatobert, testdata], ignore_index = True)
datatobert.reset_index()

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"

# tokenizer = AutoTokenizer.from_pretrained(MODEL)

dataset = load_dataset("financial_phrasebank", "sentences_allagree")

dataset = dataset['train']

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [43]:
tokenized_datasets

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'sentence'],
    num_rows: 2264
})

In [49]:
from sklearn.model_selection import train_test_split

tf_dataset = tokenized_datasets.remove_columns(["sentence"]).with_format("tensorflow")

tf_train_dataset, tf_test_dataset = train_test_split(tf_dataset, test_size=0.2)

In [52]:
import tensorflow as tf

train_features = {x: tf_train_dataset[x].to_tensor() for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_test_dataset[x].to_tensor() for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_test_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request


MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# TF
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)


In [20]:

datatobert['Scores'] = None
for index,sentence in datatobert['News'].items():
  encoded_input = tokenizer(sentence, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
  #print(labels[ranking[0]],scores[ranking[0]])
  datatobert.at[index,'Scores'] = labels[ranking[0]]


datatobert.to_csv('BERT_sentitrain_data.csv')
!cp BERT_sentitrain_data.csv "/content/drive/My Drive/"

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

In [ ]:
model.save_pretrained("/content/drive/My Drive/BERTPOSTTRAINED")


IF YOU SAVED YOUR MODEL YOU CAN NOW LOAD MODEL AND CLASSIFY YOUR DATASET

In [68]:
from transformers import AutoModelForSequenceClassification
bertpostmodel = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/BERTPOSTTRAINED", from_tf=True)


All TF 2.0 model weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.


In [ ]:
testdata['Scores'] = None
for index,sentence in testdata['Tweets'].items():
  encoded_input = tokenizer(sentence, return_tensors='tf')
  print(encoded_input)
  output = bertpostmodel.predict(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
  #print(labels[ranking[0]],scores[ranking[0]])
  testdata.at[index,'Scores'] = labels[ranking[0]]


testdata.to_csv('BERT_senti_data.csv')
!cp BERT_senti_data.csv "/content/drive/My Drive/"

In [ ]:
result = sentimentanalyzer(data['Tweets'])

In [ ]:
data_preprocessed = pd.read_csv(os.path.join('/content/drive/My Drive/','classified_test_data.csv'),encoding='latin-1')


1.   positive sentiment: compound score >= 0.05
2.   neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
3.   negative sentiment: compound score <= -0.05






In [ ]:
analyzer = SentimentIntensityAnalyzer()
listVaderSentiment = []
for sentence in data_preprocessed['Tweets']:
  vs = analyzer.polarity_scores(sentence)
  if vs.get('compound') >=0.05:
    listVaderSentiment.append('positive')
  elif vs.get('compound') > -0.05 and vs.get('compound') < 0.05:
    listVaderSentiment.append('neutral')
  elif vs.get('compound') <= -0.05:
    listVaderSentiment.append('negative')

data_preprocessed['VaderSentiment'] = listVaderSentiment

In [ ]:
data_preprocessed.to_csv('VADERclassified_test_data.csv')
!cp VADERclassified_test_data.csv "/content/drive/My Drive/"

ROZDELNIE DAT na ZACIATKU PROJEKTU po pridani STOCK dat

In [ ]:
class_topic = topic_model.topics_per_class(sentences, topics, data['Sentiment'].values)
topic_model.visualize_topics_per_class(class_topic)


In [ ]:
data['Sequence'] = None
data['All-info'] = None

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)
for index,sentence in sentences.iteritems():
  tokenized_sequence= bert_tokenizer.encode_plus(sentence,add_special_tokens = True,return_attention_mask = True)
  data.at[index,'Sequence'] = tokenized_sequence['input_ids']
  data.at[index,'All-info'] = tokenized_sequence
